# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 31 2022 1:38PM,245447,10,Stone8467404,Acute Outpatient Solutions,Released
1,Aug 31 2022 1:38PM,245446,10,Stone8467405,Acute Outpatient Solutions,Released
2,Aug 31 2022 1:37PM,245445,19,ADV80005387,"AdvaGen Pharma, Ltd",Released
3,Aug 31 2022 1:37PM,245445,19,ADV80005388,"AdvaGen Pharma, Ltd",Released
4,Aug 31 2022 1:37PM,245445,19,ADV80005389,"AdvaGen Pharma, Ltd",Released
5,Aug 31 2022 1:37PM,245445,19,ADV80005390,"AdvaGen Pharma, Ltd",Released
6,Aug 31 2022 1:37PM,245445,19,ADV80005391,"AdvaGen Pharma, Ltd",Released
7,Aug 31 2022 1:37PM,245445,19,ADV80005392,"AdvaGen Pharma, Ltd",Released
8,Aug 31 2022 1:37PM,245445,19,ADV80005393,"AdvaGen Pharma, Ltd",Released
9,Aug 31 2022 1:37PM,245445,19,ADV80005394,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,245443,Released,9
26,245444,Released,3
27,245445,Released,19
28,245446,Released,1
29,245447,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245443,NaN,NaN,9.0
245444,NaN,NaN,3.0
245445,NaN,NaN,19.0
245446,NaN,NaN,1.0
245447,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,20.0,45.0,1.0
245403,3.0,9.0,10.0
245411,0.0,0.0,1.0
245414,0.0,0.0,1.0
245415,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,20,45,1
245403,3,9,10
245411,0,0,1
245414,0,0,1
245415,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,20,45,1
1,245403,3,9,10
2,245411,0,0,1
3,245414,0,0,1
4,245415,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,20,45,1
1,245403,3,9,10
2,245411,,,1
3,245414,,,1
4,245415,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 31 2022 1:38PM,245447,10,Acute Outpatient Solutions
1,Aug 31 2022 1:38PM,245446,10,Acute Outpatient Solutions
2,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd"
21,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.
27,Aug 31 2022 1:34PM,245420,20,Alumier Labs Inc.
28,Aug 31 2022 1:31PM,245444,12,"Uniderm USA, Inc"
31,Aug 31 2022 1:14PM,245443,10,ISDIN Corporation
40,Aug 31 2022 12:21PM,245438,10,Beach Products Inc
41,Aug 31 2022 12:17PM,245436,10,Bio-PRF
45,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 31 2022 1:38PM,245447,10,Acute Outpatient Solutions,,,1
1,Aug 31 2022 1:38PM,245446,10,Acute Outpatient Solutions,,,1
2,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd",,,19
3,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,,,6
4,Aug 31 2022 1:34PM,245420,20,Alumier Labs Inc.,,,1
5,Aug 31 2022 1:31PM,245444,12,"Uniderm USA, Inc",,,3
6,Aug 31 2022 1:14PM,245443,10,ISDIN Corporation,,,9
7,Aug 31 2022 12:21PM,245438,10,Beach Products Inc,,,1
8,Aug 31 2022 12:17PM,245436,10,Bio-PRF,,3,1
9,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 1:38PM,245447,10,Acute Outpatient Solutions,1,,
1,Aug 31 2022 1:38PM,245446,10,Acute Outpatient Solutions,1,,
2,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd",19,,
3,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,6,,
4,Aug 31 2022 1:34PM,245420,20,Alumier Labs Inc.,1,,
5,Aug 31 2022 1:31PM,245444,12,"Uniderm USA, Inc",3,,
6,Aug 31 2022 1:14PM,245443,10,ISDIN Corporation,9,,
7,Aug 31 2022 12:21PM,245438,10,Beach Products Inc,1,,
8,Aug 31 2022 12:17PM,245436,10,Bio-PRF,1,3,
9,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 1:38PM,245447,10,Acute Outpatient Solutions,1,,
1,Aug 31 2022 1:38PM,245446,10,Acute Outpatient Solutions,1,,
2,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd",19,,
3,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,6,,
4,Aug 31 2022 1:34PM,245420,20,Alumier Labs Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 1:38PM,245447,10,Acute Outpatient Solutions,1.0,NaN,NaN
1,Aug 31 2022 1:38PM,245446,10,Acute Outpatient Solutions,1.0,NaN,NaN
2,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd",19.0,NaN,NaN
3,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,6.0,NaN,NaN
4,Aug 31 2022 1:34PM,245420,20,Alumier Labs Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 1:38PM,245447,10,Acute Outpatient Solutions,1.0,0.0,0.0
1,Aug 31 2022 1:38PM,245446,10,Acute Outpatient Solutions,1.0,0.0,0.0
2,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd",19.0,0.0,0.0
3,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,6.0,0.0,0.0
4,Aug 31 2022 1:34PM,245420,20,Alumier Labs Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2208901,16.0,29.0,1.0
12,245444,3.0,0.0,0.0
15,490365,30.0,46.0,20.0
19,245445,19.0,0.0,0.0
20,736245,17.0,9.0,3.0
21,490829,2.0,0.0,0.0
22,981719,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2208901,16.0,29.0,1.0
1,12,245444,3.0,0.0,0.0
2,15,490365,30.0,46.0,20.0
3,19,245445,19.0,0.0,0.0
4,20,736245,17.0,9.0,3.0
5,21,490829,2.0,0.0,0.0
6,22,981719,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,16.0,29.0,1.0
1,12,3.0,0.0,0.0
2,15,30.0,46.0,20.0
3,19,19.0,0.0,0.0
4,20,17.0,9.0,3.0
5,21,2.0,0.0,0.0
6,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,12,Released,3.0
2,15,Released,30.0
3,19,Released,19.0
4,20,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,1.0,0.0,20.0,0.0,3.0,0.0,0.0
Executing,29.0,0.0,46.0,0.0,9.0,0.0,0.0
Released,16.0,3.0,30.0,19.0,17.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,1.0,0.0,20.0,0.0,3.0,0.0,0.0
1,Executing,29.0,0.0,46.0,0.0,9.0,0.0,0.0
2,Released,16.0,3.0,30.0,19.0,17.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,1.0,0.0,20.0,0.0,3.0,0.0,0.0
1,Executing,29.0,0.0,46.0,0.0,9.0,0.0,0.0
2,Released,16.0,3.0,30.0,19.0,17.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()